In [2]:
import pandas as pd

In [3]:
demand = pd.read_excel('dataset/a6_booster_delivery_planning.xlsx')
events = pd.read_excel('dataset/ariane_tracks_booster_container_1.0.xlsx')

In [4]:
demand.head(25)

,Date,Config
0,2021-01-12,2
1,2021-08-24,4
2,2021-10-19,2
3,2022-01-28,2
4,2022-06-21,4
5,2022-10-29,4
6,2022-11-20,2
7,2023-02-27,2
8,2023-05-17,4
9,2023-07-05,2


In [6]:
events.head()

,Container,Zone,Entry,Exit
0,SN7,A,09.12.2020,24.12.2020
1,SN7,B,24.12.2020,29.12.2020
2,SN7,C,29.12.2020,30.12.2020
3,SN7,D,10.01.2021,12.01.2021
4,SN7,E,12.01.2021,27.01.2021


In [7]:
print(demand.shape)
print(events.shape)

(39, 2)
(368, 4)
